# Multi-Agent Workflow:

## 1. User Request
Everything starts with a simple prompt (e.g., *“Visualize monthly sales as a bar chart”*).  
This becomes the seed for the pipeline.

---

## 2. Planner Agent 🏗️
- Role: Creates a step-by-step plan.  
- Output: Clear instructions, no code.  
- Purpose: Prevents confusion and ensures clarity before coding.

---

## 3. Code Writer Agent 👩‍💻
- Role: Converts plan into `{code_language}` code.  
- Output: Executable code snippets.  
- Purpose: Provides the technical implementation.

---

## 4. Code Executor Agent ⚙️
- Role: Runs the generated code locally.  
- Output: Results (plots, tables, files, console logs).  
- Purpose: Turns written code into tangible outcomes.

---

## 5. Debugger Agent 🐞
- Role: Detects and fixes runtime errors.  
- Output: Corrected code, re-executed if needed.  
- Purpose: Ensures robustness and reliability.

---

## 6. Process Completion Agent 📜
- Role: Summarises what happened and nudges next steps.  
- Output: Human-readable summary.  
- Purpose: Closes the loop and guides the user forward.

---

## 7. Agent Aligner 🚦
- Role: Oversees flow of agents.  
- Rules: Plan → Confirm → Write → Confirm → Execute.  
- Purpose: Maintains order, prevents loops, ensures safety.

---

## 8. Full Workflow 🎯
1. User request → Planner.  
2. Plan confirmed → Code Writer.  
3. Code confirmed → Executor.  
4. Errors → Debugger → back to Executor.  
5. Final output → Completion Agent.  
6. Orchestrated by the Aligner.

---

## 9. Why It Works
- **Predictable**: Strict sequence.  
- **Clear**: One role per agent.  
- **Scalable**: Easy to add new agents.  
- **Safe**: Prevents accidental execution.

---

### Bottom Line
This system is like a well-managed team:  
- Planner = architect  
- Writer = engineer  
- Executor = operator  
- Debugger = troubleshooter  
- Completion = storyteller  
- Aligner = conductor  

Together, they transform vague intent into reliable results.


In [ ]:
! pip install ag2[openai]

In [ ]:

from datetime import datetime

import autogen

# from autogen.agentchat.contrib.society_of_mind_agent import SocietyOfMindAgent
from autogen import (
    AssistantAgent,
    GroupChat,
    GroupChatManager,
)

today_date = datetime.now()
from typing import Any

In [ ]:
config_list = autogen.config_list_from_json("/Users/lokesh/Desktop/code/config/groq_gpt_oss_120.json")
llm_config = {
    "timeout": 600,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0.15,
}
code_language = "Python"
location = "/Users/lokesh/Desktop/code/MCP_test/software_eng/files/"
chart_location = "/Users/lokesh/Desktop/code/MCP_test/software_eng/files/"

### Quick Recap of What Each Comment Block Does

| Agent / Block         | Responsibility                                                                 |
|------------------------|-------------------------------------------------------------------------------|
| `planner_agent`        | Creates an exhaustive, step-by-step plan (**no code**).                       |
| `code_writer`          | Turns the plan into executable `{code_language}` code.                        |
| `code_executor`        | Runs the code locally in the specified working directory.                     |
| `debugger`             | Detects runtime errors, suggests fixes, and re-executes if needed.            |
| `process_completion`   | Summarises outcomes and nudges the user forward.                              |

---


In [ ]:
planner_agent = AssistantAgent(
    name="planner_agent",
    llm_config=llm_config,
    system_message=f"""You are a Planner Agent. Your job is to understand user requests related to data analysis and create a step-by-step plan in english to achieve the desired outcome. Always read the data from the location provided.
    The Step by step plan should be exhaustive and would have all the steps which would be needed to solve the problem. If possible try to write an alternative approach to the problem and let other agents decide how they want to solve it.
    - Always save the plot with plt.save and save the plot in this location {chart_location},
     Clearly indicate which agent should be responsible for each step.
     Consider the type of analysis requested (basic, analytics, forecasting, AI/ML) and plan accordingly.
     **DO NOT GENERATE CODE YOURSELF.** Instruct the CodeWriter to generate the necessary code for each step.""",
)


code_writer = autogen.AssistantAgent(
    name="CodeWriter",
    llm_config=llm_config,
    system_message=f"""You are a {code_language} CodeWriter Agent. You receive instructions from the Planner and generate code in {code_language} to perform data analysis tasks.
            - The code should be functional and complete.
            - Specify the programming language {code_language} using code blocks (```{code_language} ... ```).
            - Use subprocess.popen  to do !pip install any module.
            - Use appropriate libraries based on the task (pandas for general analysis, scikit-learn for ML, etc.).
            - Always save the plot with plt.save and save the plot in this location {chart_location},
            """,
)


code_executor = autogen.UserProxyAgent(
    name="CodeExecutor",
    human_input_mode="NEVER",
    code_execution_config={
        "executor": autogen.coding.LocalCommandLineCodeExecutor(work_dir=location, timeout=3000),
    },
    llm_config=llm_config,
)


debugger = autogen.AssistantAgent(
    name="Debugger",
    max_consecutive_auto_reply=5,
    llm_config=llm_config,
    system_message=f"""You are a Debugger Agent. Your role is to analyze code errors reported by the CodeExecutor, suggest fixes to the CodeWriter, and verify if the fixes resolve the issues
    - Clearly identify the error, its location, and possible causes.
    - Suggest specific code modifications to the CodeWriter.
    - Use subprocess.popen  to do !pip install any module.
    - Always save the plot with plt.save and save the plot in this location {chart_location}""",
)


process_completion = AssistantAgent(
    name="process_completion",
    llm_config=llm_config,
    description="Let group manager know that the process is completed. ",
    system_message="""Respond back with tabular format for sequential info.
    Always provide the tabular response in Markdown. For example data head should be shown in markdown and so all the tabular information should be processed in markdown only Sequential information should be provided with complete information.
    Also try to provide tips for better process for example - in machine learning provide them on better model training. Tips can be from model training, evaluation, feature engineering , Exploratory data analysis Also recommend two new question to the user so that the conversation goes on. """,
)

## Why the Agent Aligner is Critical for a Multi-Agent System

| Aspect                | Problem it Solves                                                                 | How the Aligner Handles It                                                                 |
|------------------------|-----------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------|
| Conversation coherence | Conversations with multiple agents can easily drift or repeat tasks.              | Tracks every agent that has already spoken and never calls the same agent twice in a row unless explicitly requested. |
| Intent detection       | Users sometimes give vague or ambiguous requests (“show output”, “continue”).     | Analyses the full chat history, classifies the request into a workflow stage (planning, coding, execution, confirmation, etc.), and routes the message to the correct specialist agent. |
| Workflow enforcement   | The process must follow a strict order: **Plan → Confirm → Write → Confirm → Execute**. | Contains a `workflow_dependencies` rule-set that guarantees each step is only triggered when prerequisites are satisfied (e.g., code only runs after a confirmed plan). |
| Error handling & rollback | If a plan is rejected or code fails, the system must revert to an earlier stage. | Automatically returns to the appropriate previous agent (e.g., back to `planner_agent` if a plan is rejected). |
| Resource efficiency    | Running code without a proper plan wastes compute and can produce misleading results. | Checks that executable code exists and has been confirmed before routing to `code_executor_agent`. |
| Scalability            | Adding new agents or new workflow rules can become messy.                         | Decision logic is modular—new rules or agents can be added to the `decision_framework` without touching other components. |
| Security & compliance  | Ensures only vetted code is executed.                                             | By requiring explicit confirmation at each stage, the Aligner acts as a gatekeeper that prevents accidental execution of unapproved code. |

---

### Core Responsibilities

- **Context Analysis** – Scrutinises the entire chat, user intent, and past agent outputs.  
- **Agent Selection** – Picks the single most appropriate agent (`planner`, `writer`, or `executor`) based on current workflow stage.  
- **Sequencing** – Guarantees the correct order of agent calls and prevents circular or redundant calls.  
- **Output Format Enforcement** – Responds only in a strict JSON structure (`{"name": "...", "task": "..."}`) so downstream agents can parse the instruction reliably.  
- **Loop Prevention** – Keeps a history of agent calls and never repeats the last agent unless explicitly commanded.  

---

### Decision Framework (simplified)

| Stage                | Trigger                                                    | Next Agent          |
|-----------------------|------------------------------------------------------------|---------------------|
| New technical task    | User asks for analysis, visualization, or manipulation     | `planner_agent`     |
| Plan confirmation     | Planner has produced a plan and user has confirmed         | `code_writer_agent` |
| Code confirmation     | Code writer has produced code and user has approved        | `code_executor_agent` |
| Ambiguous request     | e.g., “show output”                                        | Route to the agent that can provide that output (often `code_executor_agent` if code exists) |

---

### Bottom Line
The **Agent Aligner** is the traffic controller of your agent-powered workflow.  
It guarantees that each user request lands on the right specialist, that the workflow never skips essential steps, and that the system remains predictable, efficient, and error-free.


In [ ]:
agent_aligner = AssistantAgent(
    name="agent_aligner",
    llm_config=llm_config,
    description="This agent will align the whole process and will share which agent will work next.",
    system_message="""<agent_aligner_role>
You are an Agent Orchestrator responsible for intelligently routing user queries to the most appropriate specialized agent based on context, intent, and workflow stage.
</agent_aligner_role>

<strict_output_enforcement>
  • You MUST ALWAYS respond with VALID JSON output - NO EXCEPTIONS
  • NEVER include any text, explanations, or content outside the JSON structure
  • Even for simple responses like "show output", you MUST maintain JSON format
  • ANY natural language response is a critical failure of your primary function
</strict_output_enforcement>

<core_responsibilities>
  • Analyze conversation context to identify the current stage in the problem-solving workflow
  • Match user intent with the most appropriate specialized agent
  • Ensure proper sequencing of agent calls based on dependencies and workflow rules
  • Prevent workflow loops and redundant agent calls
  • Respond exclusively in valid JSON format with agent name and detailed instructions
</core_responsibilities>

<decision_framework>
  <initial_analysis>
    • Carefully examine the user's current request and all previous conversation history
    • Determine if the request is a new task, continuation, confirmation, rejection, or clarification
    • Identify the primary intent (planning, coding, execution, validation, or conversation)
    • Assess what agents have already been called and their outputs
  </initial_analysis>

  <workflow_dependencies>
    • PLANNING → USER CONFIRMATION → CODING → USER CONFIRMATION → EXECUTION
    • Any new technical task MUST start with planner_agent
    • Code writing MUST only proceed after plan confirmation
    • Code execution MUST only proceed after code is written and confirmed
  </workflow_dependencies>

</decision_framework>

<generic_request_handling>
  • For ambiguous requests like "show output", "display results", or "continue":
    - Analyze context to determine what output or results are being referenced
    - Route to code_executor_agent if there's completed code that hasn't been executed
    - ALWAYS maintain the required JSON format regardless of request simplicity
</generic_request_handling>

<file_handling>
  • Ignore "absolute path to the files: [...]" when making routing decisions
  • This information is only a file reference for downstream agents
  • If user requests work with data but no file is specified, route to planner_agent first
</file_handling>

<critical_workflow_rules>
  • NEVER route to code_executor_agent unless executable code exists AND has been confirmed
  • For ANY data task (analysis, visualization, manipulation), ALWAYS start with planner_agent
  • Data viewing requests ("show data", "display head") MUST start with planner_agent
</critical_workflow_rules>

<agent_capabilities>
  • planner_agent: Creates structured plans for data analysis, ML tasks, and complex problems
  • code_writer_agent: Writes Python code based on confirmed plans or requirements
  • code_executor_agent: Runs completed and confirmed code
</agent_capabilities>

<intelligent_routing>
  • New technical tasks → planner_agent
  • After plan confirmation → code_writer_agent
  • After code confirmation → code_executor_agent
</intelligent_routing>

<verification_checks>
  • Before routing to code_writer_agent: Verify plan exists AND has been confirmed
  • Before routing to code_executor_agent: Verify code exists AND has been confirmed
  • Before any implementation step: Verify user has explicitly approved the preceding step
  • When unsure: Default to more conservative earlier workflow stage
</verification_checks>

<loop_prevention>
  • Track the sequence of previous agent calls, NEVER CALL THE PREVIOUS AGENT
  • Never call the same agent twice in succession unless explicitly requested
  • If user rejects a plan/code: Return to the appropriate earlier stage agent
  CHOOSE AGENT WISELY
</loop_prevention>

<output_format>
  • Your ONLY acceptable output format is the following JSON structure:
  {
      "name": "agent_name",
      "task": "Detailed instruction of what the agent should do based on current context"
  }
  • NEVER include any text before or after this JSON object
  • NEVER include explanations about why you chose a particular agent
  • NEVER use Markdown formatting for the JSON output
  • Ensure the JSON object is properly formatted with quotes around keys and string values
</output_format>""",
)

# For better responses and orchestration alignment use response template with pydantic 

## Custom speaker selection aligned with auto pickup of the agents

In [ ]:
def custom_speaker_selection_func(last_speaker: Any, groupchat: Any) -> Any:
    messages = groupchat.messages
    # file_path_provided = any("file path" in msg["content"].lower() for msg in messages)

    if len(messages) == 1:
        return groupchat.agent_by_name("agent_aligner")
    if last_speaker.name == "agent_aligner":
        if "planner_agent" in messages[-1]["content"]:
            return groupchat.agent_by_name("planner_agent")
        elif "code_writer_agent" in messages[-1]["content"]:
            return groupchat.agent_by_name("CodeWriter")
        elif "code_executor_agent" in messages[-1]["content"]:
            return groupchat.agent_by_name("CodeExecutor")
        elif "Proccesscompletion_Agent" in messages[-1]["content"]:
            return groupchat.agent_by_name("process_completion")

    if last_speaker.name in ["planner_agent", "Debugger"]:
        return groupchat.agent_by_name("agent_aligner")
    if last_speaker.name == "CodeWriter":
        return groupchat.agent_by_name("CodeExecutor")
    if last_speaker.name == "CodeExecutor":
        if "exitcode: 0" in messages[-1]["content"]:
            return groupchat.agent_by_name("process_completion")
        else:
            return groupchat.agent_by_name("Debugger")

    if last_speaker.name == "Debugger":
        return groupchat.agent_by_name("CodeWriter")

    return None

In [ ]:
cs_groupchat = GroupChat(
    [
        planner_agent,
        code_writer,
        code_executor,
        agent_aligner,
        debugger,
        process_completion,
    ],
    speaker_selection_method=custom_speaker_selection_func,
    messages=[],
    max_round=500,
)


cs_manager = GroupChatManager(cs_groupchat, llm_config=config_list[0])

In [ ]:
question = """
read the file form file path ---> '/Users/lokesh/Desktop/code/Newton/Newton_data/data/dermatology.csv'
perform univariate analysis of the data
"""
response = cs_manager.initiate_chat(cs_manager, message=question)